grasp

In [12]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
gs.init(backend=gs.gpu, seed=42)
adapter = GenesisAdapter()


[Genesis] [18:13:11] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [18:13:11] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [18:13:11] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [18:13:11] [WARNING] No Intel XPU device available. Falling back to CPU for torch device.


/home/vahid/envs/panda/lib/python3.10/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


[Genesis] [18:13:11] [INFO] Running on [11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz] with backend gs.vulkan. Device memory: 31.05 GB.
[Genesis] [18:13:11] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: 42, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [18:13:12] [INFO] Scene <7371021> created.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
)
cube_size = (0.1, 0.07, 0.03)
cube_pos = (0.7, 0, 0.015)
cube = scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = cube_pos,
    ),
    surface=gs.surfaces.Default(
        color=(0.8, 0.1, 0.1),

    ),
)

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
)

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )


[Genesis] [18:17:12] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <2c67fc2>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [18:17:12] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <ae292dc>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [18:17:12] [WARNING] (MJCF) Tendon not supported
[Genesis] [18:17:12] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [18:17:12] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [18:17:12] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [18:17:12] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [18:17:12] [INFO] Addi

In [4]:

scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

# set control gains
# Note: the following values are tuned for achieving best behavior with Franka
# Typically, each new robot would have a different set of parameters.
# Sometimes high-quality URDF or XML file would also provide this and will be parsed.
franka.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
franka.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
franka.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
)

# render rgb, depth, segmentation, and normal
# rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)


[Genesis] [18:17:19] [INFO] Building scene <7371021>...
[Genesis] [18:17:20] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [18:17:20] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/08/25 18:17:21.098 35590] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [18:17:27] [INFO] Compiling simulation kernels...
[Genesis] [18:19:06] [INFO] Building visualizer...
[Genesis] [18:19:08] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [6]:
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

franka.ready_qpos = qr
franka.set_qpos(franka.ready_qpos)
franka.control_dofs_position(franka.ready_qpos)
end_effector = franka.get_link('hand')
scene.step()

[Genesis] [18:20:27] [INFO] Running at 0.26 FPS.


In [18]:
target_entity.set_pos(end_effector.get_pos())
target_entity.set_quat(end_effector.get_quat())
scene.step()

[Genesis] [19:08:10] [INFO] Running at 0.01 FPS.


In [29]:
target_quat = np.array([0, 1, 0, 0])
center = np.array([0.5, 0, 0.5])
r = 0.1
damping = 1e-6
diag = damping * np.eye(6)
target_pos = center + np.array([1, 0, 0]) * r
init_q = franka.inverse_kinematics(
    link = end_effector,
    pos  = target_pos,
    quat = target_quat,
)
franka.set_qpos(init_q)
franka.control_dofs_position(init_q)
for i in range(100):
    target_entity.set_qpos(np.concatenate([target_pos, target_quat]))
    scene.step()

for i in range(0, 500):
    target_pos = center + np.array([np.cos(i / 360 * np.pi), np.sin(i / 360 * np.pi), 0]) * r

    target_entity.set_qpos(np.concatenate([target_pos, target_quat]))

    # Position error.
    error_pos = target_pos - end_effector.get_pos().cpu().numpy()

    # Orientation error.
    ee_quat = end_effector.get_quat().cpu().numpy()
    error_quat = gs.transform_quat_by_quat(gs.inv_quat(ee_quat), target_quat)
    error_rotvec = gs.quat_to_rotvec(error_quat)

    error = np.concatenate([error_pos, error_rotvec])

    # jacobian
    jac = franka.get_jacobian(link=end_effector).cpu().numpy()
    dq = jac.T @ np.linalg.solve(jac @ jac.T + diag, error)
    q = franka.get_qpos().cpu().numpy() + dq

    # control
    franka.control_dofs_position(q)
    scene.step()


[Genesis] [19:50:13] [INFO] Running at 0.44 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.47 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.49 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.52 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.54 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.57 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.60 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.63 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.66 FPS.
[Genesis] [19:50:13] [INFO] Running at 0.70 FPS.
[Genesis] [19:50:14] [INFO] Running at 0.73 FPS.
[Genesis] [19:50:14] [INFO] Running at 0.77 FPS.
[Genesis] [19:50:14] [INFO] Running at 0.81 FPS.
[Genesis] [19:50:14] [INFO] Running at 0.85 FPS.
[Genesis] [19:50:14] [INFO] Running at 0.89 FPS.
[Genesis] [19:50:14] [INFO] Running at 0.93 FPS.
[Genesis] [19:50:14] [INFO] Running at 0.98 FPS.
[Genesis] [19:50:14] [INFO] Running at 1.03 FPS.
[Genesis] [19:50:14] [INFO] Running at 1.08 FPS.
[Genesis] [19:50:14] [INFO] Running at 1.13 FPS.
[Genesis] [19:50:14]

In [16]:
T = sm.SE3(0.3, 0, 0.015) * sm.SE3.RPY(0, 0, np.pi)
adapter.transform(target_entity, T.A)

In [17]:
scene.step()


[Genesis] [18:44:32] [INFO] Running at 0.01 FPS.


In [7]:

# get the end-effector link
end_effector = franka.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[0]/2 + 0.02, 0, 0.11 + 0.1])
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[-2:] = 0.04
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)

[W 05/06/25 20:46:11.611 15714] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [20:46:22] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [20:46:22] [INFO] Path solution found successfully.


In [8]:
# execute the planned path
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()


[Genesis] [20:46:37] [INFO] Running at 0.28 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.29 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.31 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.33 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.34 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.36 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.38 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.40 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.42 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.44 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.47 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.49 FPS.
[Genesis] [20:46:37] [INFO] Running at 0.52 FPS.
[Genesis] [20:46:38] [INFO] Running at 0.54 FPS.
[Genesis] [20:46:38] [INFO] Running at 0.57 FPS.
[Genesis] [20:46:38] [INFO] Running at 0.60 FPS.
[Genesis] [20:46:38] [INFO] Running at 0.63 FPS.
[Genesis] [20:46:38] [INFO] Running at 0.66 FPS.
[Genesis] [20:46:38] [INFO] Running at 0.70 FPS.
[Genesis] [20:46:38] [INFO] Running at 0.74 FPS.
[Genesis] [20:46:38]

In [70]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = end_effector,
    pos  = side_box,
    quat = np.array([0, 1, 0, 0]),
)
path = franka.plan_path(
    qpos_goal     = qpos_reach,
    num_waypoints = 100, # 1s duration
)

for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

[Genesis] [14:08:21] [INFO] Path solution found successfully.
[Genesis] [14:08:21] [INFO] Running at 0.13 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.13 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.14 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.15 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.16 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.17 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.17 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.18 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.19 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.20 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.21 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.22 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.24 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.25 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.26 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.28 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.29 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.31 FPS.
[Genesis] [14:08:21] [INFO] Running at 0.32 FPS.
[Genesi

In [71]:



# grasp
franka.control_dofs_position(qpos_reach[:-2], motors_dof)
franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

for i in range(100):
    scene.step()


[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.00 FPS.
[Genesis] [16:50:51] [INFO] Running at 0.01 FPS.
[Genesis] [16:50:52] [INFO] Running at 0.01 FPS.
[Genesis] [16:50:52]

In [ ]:

# lift
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(200):
    scene.step()

In [21]:
# open
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

[Genesis] [20:51:35] [INFO] Running at 0.32 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.34 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.36 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.37 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.39 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.41 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.44 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.46 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.48 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.51 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.53 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.56 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.59 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.62 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.66 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.69 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.73 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.76 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.80 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.84 FPS.
[Genesis] [20:51:36]

[Genesis] [21:31:10] [INFO] Running at 0.06 FPS.


In [24]:
franka.get_dofs_force()

tensor([-0.0103,  0.0048,  0.0108, -0.0005, -0.0040, -0.0004, -0.0068,  0.0004,
        -0.0004], device='cuda:0')

In [39]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

cube_pos, cube_quat

(array([6.4999902e-01, 2.9570606e-06, 1.9892190e-02], dtype=float32),
 array([-1.3834608e-06,  9.9999994e-01,  4.4306205e-05,  7.9489763e-07],
       dtype=float32))

In [41]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=cube_pos + np.array([0, 0, 0.3]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

[Genesis] [00:14:45] [INFO] Running at 0.28 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.29 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.31 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.32 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.34 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.36 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.38 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.40 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.42 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.44 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.46 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.48 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.51 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.54 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.56 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.59 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.63 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.66 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.69 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.73 FPS.
[Genesis] [00:14:46]

In [ ]:

cam.start_recording()
import numpy as np

for i in range(120):
    scene.step()
    cam.set_pose(
        pos    = (3.0 * np.sin(i / 60), 3.0 * np.cos(i / 60), 2.5),
        lookat = (0, 0, 0.5),
    )
    cam.render()
cam.stop_recording(save_to_filename='video.mp4', fps=60)